In [2]:
cd ..

d:\research\fedmiss_bench


In [24]:
import os
import itertools
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

nnn_imp_fed = {
    'simple': ['local', 'fedavg'],
    'em': ['local', 'fedavg'],
    'linear_ice': ['local', 'fedavg'],
    'missforest': ['local', 'fedtree']
}

nn_imp_fed = {
    'miwae': ['local', 'fedavg', 'fedavg_ft', 'fedprox'],
    'gain': ['local', 'fedavg', 'fedavg_ft', 'fedprox'],
}

dataset_dp = {
    'vehicle_np': ['natural-partition'],
    'school_np': ['natural-partition'],
}

dataset_metric_pred = {
    'vehicle_np': ['accuracy', 'f1', 'auc', 'prc'],
    'school_np': ['mse', 'mae', 'r2', 'msle'],
}

In [25]:
def load_ret(datasets, imputers, met):
    
    rets = []
    unfinished = {}
    for dataset in datasets:
        
        data_partition = dataset_dp[dataset]
        unfinished[dataset] = {}
        missing = ['mcar', 'mar-homog', 'mnar2-homog', 'mar-heter', 'mnar2-heter']
        models = ['nn']
        
        for imputer in imputers:
            unfinished[dataset][imputer] = set()
            print(dataset, imputer)            
            if imputer in nnn_imp_fed:
                rounds = [0,1,2,3,4,5,6,7,8,9]
                feds = nnn_imp_fed[imputer]
            else:
                rounds = [0,1,2,3,4]
                feds = nn_imp_fed[imputer]

            param = itertools.product(feds, data_partition, missing, rounds, models)
            dir_path = f'./results/raw_results/eval_results/alpha/{dataset}/'

            count, total = 0, 0
            for i, (fed, dp, ms, round, model) in enumerate(param):
                
                file_path = f'{dir_path}{dp}_{ms}/{imputer}/{fed}/{round}/{model}/eval_results.json'
                total += 1
                try:
                    with open(file_path, 'r') as f:
                        ret = json.load(f)
                        if met == 'imp':
                            metrics = ['rmse', 'sliced-ws']
                            ret_metrics = [ret['results']['imp_quality'][metric] for metric in metrics]
                        elif met == 'pred':
                            metrics = dataset_metric_pred[dataset]
                            ret_metrics = [ret['results']['pred_performance'][metric] for metric in metrics]
                        else:
                            raise ValueError('Invalid metric')
                        
                        for id, items in enumerate(zip(*ret_metrics)):
                            setting = {
                                'dataset': dataset,
                                'data_partition': dp,
                                'imputer': imputer,
                                'fed': fed,
                                'missing': ms,
                                'round': round,
                                'model': model,
                                'client_id': id
                            }
                            setting.update({metric: item for metric, item in zip(metrics, items)})
                            rets.append(setting)
                
                except Exception as e:
                    print(e)
                    count += 1
                    unfinished[dataset][imputer].add((dataset, dp, ms, imputer, fed, round, model, e))
                    continue

            print(f'{count}/{total}')
        
    return rets, unfinished

In [26]:
result_pred, unfinished = load_ret(
    ['vehicle_np', 'school_np'],
    ['simple', 'em', 'linear_ice', 'missforest', 'miwae', 'gain'], 
    met = 'pred'
)

vehicle_np simple
0/100
vehicle_np em
0/100
vehicle_np linear_ice
0/100
vehicle_np missforest
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/vehicle_np/natural-partition_mcar/missforest/local/0/nn/eval_results.json'
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/vehicle_np/natural-partition_mcar/missforest/local/1/nn/eval_results.json'
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/vehicle_np/natural-partition_mcar/missforest/local/2/nn/eval_results.json'
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/vehicle_np/natural-partition_mcar/missforest/local/3/nn/eval_results.json'
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/vehicle_np/natural-partition_mcar/missforest/local/4/nn/eval_results.json'
[Errno 2] No such file or directory: './results/raw_results/eval_results/alpha/vehicle_np/natural-partition_mcar/missforest/local/5/nn/e